# Advanced: Switches & Skeletons

In [88]:
import robotic as ry
import time

In [ ]:
C = ry.Config()
C.addFile(ry.raiPath('model2.g'))
C.addFrame('Box_Marker') .setPosition(
    C.getFrame('box').getPosition()
) .setQuaternion(
    C.getFrame('box').getQuaternion()
) .setShape(ry.ST.marker, [0.3])
C.view()

In [106]:
komo = ry.KOMO()
komo = ry.KOMO(C, 2.5, 30, 2, False)

In [107]:
komo.addControlObjective([], 2, 1e0)
komo.addQuaternionNorms([], 3.)

In [108]:
#grasp
komo.addModeSwitch([1., 2.], ry.SY.stable, ["gripper", "box"], True)
komo.addObjective([1.], ry.FS.positionDiff, ["gripper", "box"], ry.OT.eq, [1e2])
komo.addObjective([1.], ry.FS.scalarProductXX, ["gripper", "box"], ry.OT.eq, [1e2], [0])
komo.addObjective([1.], ry.FS.vectorZ, ["gripper"], ry.OT.eq, [1e2], [0., 0., 1.])
#slow - down - up
komo.addObjective([1.], ry.FS.qItself, [], ry.OT.eq, [], [], 1)
komo.addObjective([.9,1.1], ry.FS.position, ["gripper"], ry.OT.eq, [], [0.,0.,.1], 2)
#place
komo.addModeSwitch([2., -1.], ry.SY.stable, ["table", "box"])
komo.addObjective([2.], ry.FS.positionDiff, ["box", "table"], ry.OT.eq, [1e2], [0,0,.08])
komo.addObjective([2.], ry.FS.vectorZ, ["gripper"], ry.OT.eq, [1e2], [0., 0., 1.])
#slow - down - up
komo.addObjective([2.], ry.FS.qItself, [], ry.OT.eq, [], [], 1)
komo.addObjective([1.9,2.2], ry.FS.position, ["gripper"], ry.OT.eq, [], [0.,0.,.1], 2)
# Side Note (If you dont't want to use Mode Switches): You can also attach the frames using C.attach() -> Check Documentation!

In [ ]:
ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print('size of path:', q.shape)

for t in range(q.shape[0]):
    C.setJointState(q[t])
    C.view(False, f'waypoint {t}')
    time.sleep(.1)

In [ ]:
komo.view_play(False, .7)

## Same again, but only the keyframes:

In [95]:
komo=0

In [96]:
komo = ry.KOMO()
komo = ry.KOMO(C, 2.5, 30, 1, False)
komo.addControlObjective([], 1, 1e-1) # DIFFERENT!
komo.addQuaternionNorms([], 3.)

In [97]:

#grasp
komo.addModeSwitch([1., 2.], ry.SY.stable, ["gripper", "box"])
komo.addObjective([1.], ry.FS.positionDiff, ["gripper", "box"], ry.OT.eq, [1e2])
komo.addObjective([1.], ry.FS.scalarProductXX, ["gripper", "box"], ry.OT.eq, [1e2], [0.])
komo.addObjective([1.], ry.FS.vectorZ, ["gripper"], ry.OT.eq, [1e2], [0., 0., 1.])

# DIFFERENT: UP-DOWN MISSING
    
#place
komo.addModeSwitch([2., -1.], ry.SY.stable, ["table", "box"]) 
komo.addObjective([2.], ry.FS.positionDiff, ["box", "table"], ry.OT.eq, [1e2], [0,0,.08])
komo.addObjective([2.], ry.FS.vectorZ, ["gripper"], ry.OT.eq, [1e2], [0., 0., 1.])

In [ ]:
ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print('size of path:', q.shape)

komo.view(False, "result")


In [ ]:
komo.view_play(True, .9)

## Skeletons

In [ ]:
del C
del S
del komo

In [ ]:
import sys
sys.path += ['../../lib', '../../build', '../../../build']
import numpy as np
import robotic as ry
C = ry.Config()
C.addFile(ry.raiPath("model.g"))
C.view()

In [158]:
S = ry.Skeleton()

In [159]:
# S.addEntry([1.], ry.SY.topBoxGrasp, ["gripper", "box2"])
S.addEntry([1., 1.], ry.SY.touch,  ["gripper", "box2"])
S.addEntry([1., 2.], ry.SY.stable, ["gripper", "box2"])
S.addEntry([0.8, 1.2], ry.SY.downUp, ["gripper"])
S.addEntry([1.8, 2.2], ry.SY.downUp, ["gripper"])
S.addEntry([2., 2.], ry.SY.poseEq, ["box2", "target2"])
S.addEntry([2., -1], ry.SY.stable, ["table", "box2"])
# S.enableAccumulatedCollisions(True)

In [13]:
#komo = S.getKomo_waypoints(C, 0.1, 0.01, 0.0)

lenScale, homingScale, and collScale are parameters that are likely used for some kind of optimization or cost function. Here's a brief explanation of each:

    lenScale (Length Scale): This parameter is likely used to control the relative importance of the length or distance-related terms in the optimization. A higher value would mean that length-related terms have more influence on the optimization, while a lower value would reduce their impact.

    homingScale (Homing Scale): This parameter might be related to a "homing" behavior, which typically means returning to a specific position or configuration. It could control how strongly the algorithm prioritizes returning to a specific home position.

    collScale (Collision Scale): This parameter likely controls how much the optimization considers avoiding collisions. A higher value would mean that avoiding collisions is more important, while a lower value would reduce its importance.

In [ ]:
## solve for waypoints: create a komo instance, create nlp instance, then call generic solver

komo = S.getKomo_path(C, stepsPerPhase=30, accScale=1e0, lenScale=1e-2, homingScale=1e-1, collScale=1e0)

ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve() ## WHAT IS NLP!! SOLVER?
print(ret)
q = komo.getPath()
print('size of path:', q.shape)

komo.view(False, "result")
komo.view_play(True, .5)


In [ ]:
komo.view_play(True, .5)

### Alternative Method

In [ ]:
komo = S.getKomo_path(C, stepsPerPhase=30, accScale=1e0, lenScale=1e-2, homingScale=1e-1, collScale=1e0)
nlp = komo.nlp()
sol = ry.NLP_Solver()
sol.setProblem(nlp)
sol.setOptions( stopTolerance=1e-2 )
ret = sol.solve()
waypoints = komo.getPath_qAll()
# report on result, view, and play
print(ret)

In [ ]:
#komo.view(True, 'waypoints solution')
komo.view_play(True, .2)

In [ ]:
# print("costs:", komo.getCosts())
# komo.getReport()
# #komo.verbose = 4;
# komo.optimize();
# #/  komo.checkGradients();
# komo.view(False, "result")